In [1]:
import ee
ee.Initialize()
from iteru import *
from IPython.display import display
import ipywidgets
import datetime

In [2]:
Map = Map()
Map

Map(center=[27, 31], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Sea…

In [21]:
aoi = Map.aoi

In [22]:
demvis = {'min':488,'max':1863,'palette':['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']
}

In [77]:
dataset = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2')\
           .filter(ee.Filter.bounds(aoi))\
           .select('DSM')\
           .median()\
           .clip(aoi)

In [24]:
#Map.add_layer_widgets(dataset,demvis,'GRED_DEM')

In [25]:
S2 = ee.ImageCollection("COPERNICUS/S2_SR")\
     .filter(ee.Filter.bounds(aoi))\
     .filter(ee.Filter.date('2022-1-01','2022-2-1'))\
     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',2))

In [9]:
vis_param = get_vis_params(S2)

In [26]:
S2_median = S2.median().clip(aoi)

In [27]:
S2.size().getInfo()

6

In [28]:
#Map.add_layer_widgets(S2_median,vis_param,'S2_Median')

In [29]:
ndwi = S2_median.normalizedDifference(['B3', 'B8'])

In [30]:
ndwiMasked = ndwi.updateMask(ndwi.gte(0))

In [31]:
#Map.add_layer_widgets(ndwiMasked,{'min':0,'max':0.5,'palette': ['00FFFF', '0000FF']},'NDWI')

In [50]:
v = ee.Image(1).updateMask(ndwiMasked).reduceToVectors(
           geometry = ndwiMasked.geometry(),
           crs =  ndwiMasked.projection(),
           scale = 10 , 
           geometryType = 'polygon',
           eightConnected = False,
           labelProperty =  'water_cover',
           bestEffort = True
)

In [51]:
def calc_area(feature):
    area = feature.geometry().area(maxError = 1)
    
    return feature.set({'Area':area})

In [59]:
v = v.map(calc_area)

In [60]:
v = ee.Feature(v.sort('Area', False).first())

In [36]:
Map.add_layer_widgets(v,{'color':'red'},'Vector')

In [37]:
lake_dem = dataset.clip(v)

In [38]:
Map.add_layer_widgets(lake_dem,demvis,'Lake DEM')

In [39]:
ele = lake_dem.reduceRegion(
                         reducer = ee.Reducer.max(),
                         geometry = lake_dem.geometry(), 
                         scale = 10, 
                        maxPixels = 1e10)

In [40]:
water_level = ele.getInfo()['DSM']

In [41]:
water_level

604

In [42]:
import numpy as np

In [43]:
#latlng = ee.Image.pixelLonLat().addBands(lake_dem)
latlng = lake_dem.reduceRegion(reducer=ee.Reducer.toList(), geometry=v.geometry(), maxPixels=1e11, scale=30)

In [44]:
elev_values = np.array((ee.Array(latlng.get("DSM")).getInfo()),dtype=np.int64)

In [45]:
elev_values.size

418968

In [46]:
volume = water_level - elev_values[elev_values < water_level]

In [47]:
(volume.sum()*900)/1e9

17.6555745